# Generating the JetBrains Academy Course

This notebook serves as a showcase to the functionality of the second part of the pipeline to generate a JetBrains Academy Course from a closed and solved OSS GitHub issue. This part of the pipeline will create the lessons and tasks sequentially for the sake of simplicity and completeness. However, as the tasks do not depend on eachother during the file creation, several steps could be parallelized to improve the performance.

## Processing the Input Data

The first step of this part of the pipeline is to properly read the input given and pass it along to the appropriate functions. The given input is of the following form:

```json
{
    "id": 0,
    "title" : "Placeholder Exercise",
    "gitInfo" : {
        "repo" : "https://repo.com",
        "issue" : "https://repo.com/issues/0",
        "pr" : "https://repo.com/pulls/0",
    },
    "exercise" : "This is a placeholder exercise. Here is a description of the exercise.",
    "steps" : [
        {
            "id" : 0,
            "summary" : "Step 1",
            "codeSteps" : [
                {
                    "id" : 0,
                    "summary" : "Code Step 1",
                    "code" : "console.log('Step 1!')",
                    "path" : "test/test.ts",
                    "range" : {
                        "start" : 1,
                        "end" : 2,
                    }
                },
                {
                    "id" : 1,
                    "summary" : "Code Step 2",
                    "code" : "console.log('Step 2!')",
                    "path" : "test/test.ts",
                    "range" : {
                        "start" : 10,
                        "end" : 12,
                    }
                }
            ]
        },
        {
            "id" : 1,
            "summary" : "Step 2",
            "codeSteps" : [
                {
                    "id" : 0,
                    "summary" : "Code Step 1",
                    "code" : "console.log('Hello, World!')",
                    "path" : "test/test.ts",
                    "range" : {
                        "start" : 2,
                        "end" : 3,
                    }
                }
            ]
        }
    ],
}
```

In [20]:
# Dependencies of the pipeline
import os
import yaml

In [48]:
# Testing Random Shit

content = {
    "code": "console.log(\"Hello World!\")",
    "test": "",
    "config": None
}
generateTaskFolder("Testing Task", "# Testing Is Wonderful", "theory", content, "test_data")

### Task Content

To create the task folder, I expect the input to be passed as the following format:

```json
{
    "title": "Title of the task",
    "description": "Text which will be desplayed in the task description, already formatted",
    "category": "category of the task that will be specified in the config file",
    "content": {

    },
    "lesson_path": "Path of the lesson directory"
}
```

In [47]:
def generateTaskFolder(title, description, category, content, lesson_path):
    """
    This function generates a task folder which is the content of a lesson folder. 
    The inputs of the function are its title, description and type as well as its content. 
    By content we mean what the master solution of the task is.
    """

    task_path = os.path.join(lesson_path, title)
    os.makedirs(task_path, exist_ok=True)
    file_paths = {}

    # TODO: Files to create: -task.md -task.js -task-info.yaml
    taskMD_path = os.path.join(task_path, "task.md")
    with open(taskMD_path, "w", encoding="utf-8") as f:
        f.write(description)
    
    for file in content:
        if content[file] == None or content[file] == "":
            continue
        file_path = os.path.join(task_path, file)
        if file == "test":
            test_folder = os.path.join(task_path, "test")
            os.makedirs(test_folder, exist_ok=True)
            file_path = os.path.join(test_folder, file)
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(content[file])
        file_paths[elem] = file_path

    taskYML_path = os.path.join(task_path, "task-info.yaml")
    with open(taskYML_path, "w", encoding="utf-8") as f:
        file_content = {
            "type": category,
            "custom_name": title,
            "files": []
        }
        for file in file_paths:
            file_content["files"].append({"name": file, "visible": True})
            yaml.dump(file_content, f, default_flow_style=False)


In [ ]:
def generateLessonFolder(title, description, category, steps, course_path):
    """
    This function generates a lesson folder necessary for a JetBrains Academy course. Its input
    are the content of each task it has as well as its title and description
    The types of the lesson's are: "introduction", "theory", "coding", and "debrief"
    """
    
    lesson_path = os.path.join(course_path, category)
    os.makedirs(lesson_path, exist_ok=True)

    if (category == "introduction"):
        # TODO: Don't forget to generate the config.js file to properly set-up the course for the plugin

    elif (category == "theory"):

    elif (category == "coding"):

    elif (category == "debrief"):

    else:
        raise Exception("Wrong Lesson Type given to the function!")


In [ ]:
def generateTheoryContent(issue_url, pr_url):
    """
    This function takes as input the issue and pr of the exercise. Its output is the content used
    to create the Theory Lesson for the course, that presents the necessary theoretical knowledge to 
    the student taking the course. 
    """
    # TODO: Create a prompt to properly generate the content

    '''
    TODO: The output of this function needs to be of the form
    {
        "title": "",
        "tasks": {
            "title": "",
            "description": ""
        }
    }
    where the description is already in MarkDown formatting.
    '''

In [ ]:
def generateCourseFolder(input):
    """
    This function takes as input a specific json format generated by the previous part of the pipeline. 
    It outputs an entire folder which possesses the structure needed to be considered a course by 
    the JetBrains Academy plugin
    """
    # TODO: Check for the proper format of the input
    try:
        theory_content = generateTheoryContent(input.gitInfo.issue, input.gitInfo.pr)
    except:
        print("Wrong input type has been given.")

    return None

## Running the Pipeline

Now we are ready to run the entire pipeline. Feel free to run the last cell to see how a course is generated!

In [ ]:
placeholder_input = {
    "id": 0
}
generateCourseFolder(placeholder_input)